# All Weather Live Portfolio Tracker

**Start Date**: January 28, 2026 (last trading day in data)  
**Strategy**: v1.2 (Adaptive Rebalancing + Ledoit-Wolf Shrinkage)  
**Rebalance Frequency**: Weekly (Monday)  
**Drift Threshold**: 5%

---

## Instructions:
1. Run all cells to see current portfolio status
2. Check **"Next Action"** section for trades to execute
3. View **PnL Curve** starting from Jan 28, 2026
4. Rerun notebook after executing trades to update

**Note**: 511260.SH (10Y Treasury) trades in 100-share lots only

**Data Note**: If you see "Note: Using last available date" in the output below, the data has been updated. PnL tracking will start from the last date in the dataset.

In [ ]:
import sys
sys.path.append('../')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime, timezone, timedelta
import pytz

from src.data_loader import load_prices
from src.optimizer import optimize_weights
from src.utils.reporting import print_section, format_currency

plt.style.use('seaborn-v0_8-darkgrid')
plt.rcParams['figure.figsize'] = (14, 8)

print("✓ Imports successful")

## 1. Configuration & Initial Position

**EDIT THIS CELL** to set your initial positions.

The notebook will automatically adjust to use the last available date in the dataset.

In [ ]:
# ========== EDIT YOUR INITIAL POSITIONS HERE ==========
# Initial position as of January 28, 2026 (last trading day in data)
INITIAL_POSITIONS = {
    '510300.SH': 700,      # CSI 300 - Large cap stocks
    '510500.SH': 300,      # CSI 500 - Mid/small cap stocks  
    '513500.SH': 1100,     # S&P 500
    '511260.SH': 100,      # 10Y Treasury (trades in 100s)
    '518880.SH': 300,      # Gold
    '000066.SH': 0,        # China Index
    '513100.SH': 1100      # Nasdaq-100
}

INITIAL_CASH = 27354  # Remaining cash in CNY
# ======================================================

# Strategy parameters
START_DATE = '2026-01-28'  # Will auto-adjust to last available date
LOOKBACK = 252  # Days for covariance calculation
REBALANCE_THRESHOLD = 0.05  # 5% drift triggers rebalance
COMMISSION_RATE = 0.0003  # 0.03%

# Chicago timezone
CHICAGO_TZ = pytz.timezone('America/Chicago')
TODAY_CHICAGO = datetime.now(CHICAGO_TZ)

print(f"Current Time (Chicago): {TODAY_CHICAGO.strftime('%Y-%m-%d %H:%M:%S %Z')}")
print(f"\nInitial Positions (as of {START_DATE}):")
for etf, shares in INITIAL_POSITIONS.items():
    print(f"  {etf}: {shares:,.0f} shares")
print(f"  Cash: {format_currency(INITIAL_CASH)}")

## 2. Load Market Data

In [ ]:
# Load historical prices
prices = load_prices('../data/etf_prices_7etf.csv')

# Validate and adjust start date if needed
requested_start = pd.Timestamp(START_DATE)
if requested_start not in prices.index:
    # Find the closest date that exists
    if requested_start > prices.index[-1]:
        actual_start = prices.index[-1]
        print(f"Note: Requested start date {START_DATE} is beyond data range.")
        print(f"      Using last available date: {actual_start.date()}")
    else:
        # Use the previous available date
        actual_start = prices.index[prices.index <= requested_start][-1]
        print(f"Note: Requested date {START_DATE} not in data.")
        print(f"      Using closest previous date: {actual_start.date()}")
    START_DATE = actual_start.strftime('%Y-%m-%d')

# Filter to relevant period (need lookback before start date)
start_idx = prices.index.get_loc(pd.Timestamp(START_DATE))
lookback_start = prices.index[max(0, start_idx - LOOKBACK)]

print(f"\nData loaded: {len(prices)} days")
print(f"Period: {prices.index[0].date()} to {prices.index[-1].date()}")
print(f"Tracking from: {START_DATE}")
print(f"\nLatest prices (as of {prices.index[-1].date()}):")
print(prices.iloc[-1])

## 3. Calculate Current Portfolio Status

In [ ]:
def calculate_portfolio_value(positions, prices_series, cash=0):
    """Calculate total portfolio value."""
    position_value = sum(
        shares * prices_series.get(etf, 0)
        for etf, shares in positions.items()
    )
    return position_value + cash

def calculate_weights(positions, prices_series, cash=0):
    """Calculate current portfolio weights."""
    total_value = calculate_portfolio_value(positions, prices_series, cash)
    if total_value == 0:
        return {etf: 0.0 for etf in positions.keys()}
    
    weights = {}
    for etf, shares in positions.items():
        position_value = shares * prices_series.get(etf, 0)
        weights[etf] = position_value / total_value
    return weights

# Get latest prices
latest_prices = prices.iloc[-1]

# Current portfolio status
current_value = calculate_portfolio_value(INITIAL_POSITIONS, latest_prices, INITIAL_CASH)
current_weights = calculate_weights(INITIAL_POSITIONS, latest_prices, INITIAL_CASH)

print_section("Current Portfolio Status")
print(f"\nTotal Value: {format_currency(current_value)}")
print(f"Cash: {format_currency(INITIAL_CASH)}")
print(f"Invested: {format_currency(current_value - INITIAL_CASH)}")

print("\nCurrent Weights:")
for etf, weight in sorted(current_weights.items(), key=lambda x: x[1], reverse=True):
    shares = INITIAL_POSITIONS[etf]
    value = shares * latest_prices[etf]
    print(f"  {etf}: {weight:7.2%}  ({shares:>8,.0f} shares, {format_currency(value):>12s})")

## 4. Calculate Target Weights (Risk Parity)

In [ ]:
# Get historical returns for covariance calculation
hist_end_idx = prices.index.get_loc(prices.index[-1])
hist_start_idx = max(0, hist_end_idx - LOOKBACK)
hist_returns = prices.iloc[hist_start_idx:hist_end_idx].pct_change().dropna()

print(f"Using {len(hist_returns)} days of returns for optimization")
print(f"Period: {hist_returns.index[0].date()} to {hist_returns.index[-1].date()}")

# Calculate risk parity weights (v1.2 with shrinkage)
target_weights_array = optimize_weights(hist_returns, use_shrinkage=True)
target_weights = dict(zip(prices.columns, target_weights_array))

print_section("Target Weights (Risk Parity v1.2)")
print("\nOptimal allocation:")
for etf, weight in sorted(target_weights.items(), key=lambda x: x[1], reverse=True):
    print(f"  {etf}: {weight:7.2%}")

## 5. Calculate Rebalancing Trades

**Note**: 511260.SH trades in 100-share lots

In [ ]:
def calculate_drift(current_weights, target_weights):
    """Calculate maximum weight drift."""
    max_drift = 0.0
    for etf in current_weights.keys():
        drift = abs(current_weights[etf] - target_weights[etf])
        max_drift = max(max_drift, drift)
    return max_drift

def round_to_lot_size(shares, etf):
    """Round shares to appropriate lot size."""
    if etf == '511260.SH':
        # Round to nearest 100
        return round(shares / 100) * 100
    else:
        # Round to nearest whole share
        return round(shares)

def calculate_rebalance_trades(current_positions, target_weights, prices_series, total_value, cash):
    """Calculate required trades to rebalance portfolio."""
    trades = {}
    
    for etf in current_positions.keys():
        target_value = total_value * target_weights[etf]
        target_shares = target_value / prices_series[etf]
        target_shares_rounded = round_to_lot_size(target_shares, etf)
        
        current_shares = current_positions[etf]
        trade_shares = target_shares_rounded - current_shares
        
        if trade_shares != 0:
            trade_value = trade_shares * prices_series[etf]
            commission = abs(trade_value) * COMMISSION_RATE
            
            trades[etf] = {
                'current_shares': current_shares,
                'target_shares': target_shares_rounded,
                'trade_shares': trade_shares,
                'price': prices_series[etf],
                'trade_value': trade_value,
                'commission': commission,
                'side': 'BUY' if trade_shares > 0 else 'SELL'
            }
    
    return trades

# Calculate drift
drift = calculate_drift(current_weights, target_weights)
needs_rebalance = drift > REBALANCE_THRESHOLD

print_section("Rebalancing Analysis")
print(f"\nMaximum drift: {drift:.2%}")
print(f"Rebalance threshold: {REBALANCE_THRESHOLD:.2%}")
print(f"\nNeeds rebalancing: {'YES ✓' if needs_rebalance else 'NO - within threshold'}")

if needs_rebalance:
    trades = calculate_rebalance_trades(
        INITIAL_POSITIONS, 
        target_weights, 
        latest_prices, 
        current_value,
        INITIAL_CASH
    )
    
    if trades:
        print("\n" + "="*70)
        print("REQUIRED TRADES")
        print("="*70)
        
        total_commission = 0
        
        for etf, trade in trades.items():
            print(f"\n{etf}:")
            print(f"  Current: {trade['current_shares']:,.0f} shares")
            print(f"  Target:  {trade['target_shares']:,.0f} shares")
            print(f"  Action:  {trade['side']} {abs(trade['trade_shares']):,.0f} shares @ ¥{trade['price']:.4f}")
            print(f"  Value:   {format_currency(abs(trade['trade_value']))}")
            print(f"  Commission: {format_currency(trade['commission'])}")
            
            total_commission += trade['commission']
        
        print(f"\nTotal Commission: {format_currency(total_commission)}")
    else:
        print("\nNo trades needed (positions already aligned)")
else:
    print("\n✓ Portfolio is within drift threshold. No action needed.")

## 6. PnL Tracking (from Jan 29, 2026)

Track daily portfolio value and calculate PnL

In [ ]:
# Calculate daily portfolio values starting from START_DATE
start_date_idx = prices.index.get_loc(pd.Timestamp(START_DATE))
tracking_prices = prices.iloc[start_date_idx:]

# Calculate daily values
daily_values = []
daily_pnl = []

initial_value = calculate_portfolio_value(INITIAL_POSITIONS, prices.loc[START_DATE], INITIAL_CASH)

for date, row in tracking_prices.iterrows():
    daily_value = calculate_portfolio_value(INITIAL_POSITIONS, row, INITIAL_CASH)
    pnl = daily_value - initial_value
    pnl_pct = (pnl / initial_value * 100) if initial_value > 0 else 0
    
    daily_values.append(daily_value)
    daily_pnl.append(pnl)

pnl_df = pd.DataFrame({
    'Date': tracking_prices.index,
    'Portfolio Value': daily_values,
    'PnL': daily_pnl,
    'PnL %': [p / initial_value * 100 if initial_value > 0 else 0 for p in daily_pnl]
}).set_index('Date')

print_section("PnL Summary")
print(f"\nInitial Value ({START_DATE}): {format_currency(initial_value)}")
print(f"Current Value ({tracking_prices.index[-1].date()}): {format_currency(daily_values[-1])}")
print(f"\nTotal PnL: {format_currency(daily_pnl[-1])} ({pnl_df['PnL %'].iloc[-1]:+.2f}%)")
print(f"Days tracked: {len(pnl_df)}")

if len(pnl_df) > 1:
    print(f"\nBest day: {format_currency(pnl_df['PnL'].max())} on {pnl_df['PnL'].idxmax().date()}")
    print(f"Worst day: {format_currency(pnl_df['PnL'].min())} on {pnl_df['PnL'].idxmin().date()}")

print("\nRecent PnL:")
print(pnl_df.tail())

## 7. PnL Visualization

In [ ]:
fig, (ax1, ax2) = plt.subplots(2, 1, figsize=(14, 10))

# Portfolio value over time
ax1.plot(pnl_df.index, pnl_df['Portfolio Value'], linewidth=2.5, color='#2ca02c')
ax1.axhline(y=initial_value, color='gray', linestyle='--', alpha=0.5, label='Initial Value')
ax1.set_title(f'Portfolio Value (since {START_DATE})', fontsize=14, fontweight='bold')
ax1.set_ylabel('Value (¥)', fontsize=12)
ax1.legend()
ax1.grid(True, alpha=0.3)
ax1.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'¥{x/1e6:.2f}M' if x >= 1e6 else f'¥{x:,.0f}'))

# Cumulative PnL
colors = ['green' if x >= 0 else 'red' for x in pnl_df['PnL']]
ax2.bar(pnl_df.index, pnl_df['PnL'], color=colors, alpha=0.7, width=0.8)
ax2.axhline(y=0, color='black', linestyle='-', linewidth=0.8)
ax2.set_title('Daily PnL', fontsize=14, fontweight='bold')
ax2.set_ylabel('PnL (¥)', fontsize=12)
ax2.set_xlabel('Date', fontsize=12)
ax2.grid(True, alpha=0.3, axis='y')
ax2.yaxis.set_major_formatter(plt.FuncFormatter(lambda x, p: f'¥{x:,.0f}'))

plt.tight_layout()
plt.show()

print(f"\n{'='*70}")
print(f"Last updated: {TODAY_CHICAGO.strftime('%Y-%m-%d %H:%M:%S %Z')}")
print(f"{'='*70}")

---

## Next Steps:

1. **If rebalancing needed**: Execute the trades shown in "Required Trades" section
2. **After trading**: Update `INITIAL_POSITIONS` and `INITIAL_CASH` in cell 2 with new values
3. **Rerun notebook**: To see updated status and PnL

**Rebalance Schedule**: Every Monday (or when drift > 5%)

**Remember**: 
- 511260.SH must be traded in 100-share increments
- PnL tracking starts from the date you set in cell 2
- Update positions in cell 2 after each trade session